## Training a Simple Neural Network, with PyTorch Data Loading

In [2]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

Hyperparameters

In [3]:
# A helper function to randomly initialize weights and biases
# for a dense neural network layer
def random_layer_params(m, n, key, scale=1e-2):
  w_key, b_key = random.split(key)
  return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

# Initialize all layers for a fully-connected neural network with sizes "sizes"
def init_network_params(sizes, key):
  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

layer_sizes = [784, 512, 512, 10]
step_size = 0.01
num_epochs = 8
batch_size = 128
n_targets = 10
params = init_network_params(layer_sizes, random.PRNGKey(0)) # randomly initialization

auto-batching predictions:首先，让我们定义预测函数。请注意，我们是为单个图像示例定义的。我们将使用 JAX 的 vmap 函数自动处理迷你批次，而不会影响性能。

In [4]:
from jax.scipy.special import logsumexp

def relu(x):
  return jnp.maximum(0, x)

def predict(params, image):
  # per-example predictions
  activations = image
  for w, b in params[:-1]:
    outputs = jnp.dot(w, activations) + b
    activations = relu(outputs)
  
  final_w, final_b = params[-1]
  logits = jnp.dot(final_w, activations) + final_b # 通常，神经网络的最后一层是 softmax 层，用于将模型的原始输出转化为概率分布。
  return logits - logsumexp(logits)

刚刚的是针对单个图像定义的，先检查一下是否对单可用且不能对batch

In [5]:
# This works on single examples
random_flattened_image = random.normal(random.PRNGKey(1), (28 * 28,))
preds = predict(params, random_flattened_image)
print(preds.shape)

# Doesn't work with a batch
random_flattened_images = random.normal(random.PRNGKey(1), (10, 28 * 28))
try:
  preds = predict(params, random_flattened_images)
except TypeError:
  print('Invalid shapes!')

(10,)
Invalid shapes!


然后看看对于batch的情况使用vmap批量处理

In [6]:
# Let's upgrade it to handle batches using `vmap`

# Make a batched version of the `predict` function
batched_predict = vmap(predict, in_axes=(None, 0))

# `batched_predict` has the same call signature as `predict`
batched_preds = batched_predict(params, random_flattened_images)
print(batched_preds.shape)

(10, 10)


至此，我们已经具备了定义神经网络并对其进行训练所需的所有要素。我们已经构建了一个自动批处理版本的 predict，我们应该可以在损失函数中使用它。我们应该可以使用 grad 求出损失相对于神经网络参数的导数。最后，我们应该可以使用 jit 来加速一切。

Utility and loss functions

In [7]:
def one_hot(x, k, dtype=jnp.float32):
  """Create a one-hot encoding of x of size k."""
  return jnp.array(x[:, None] == jnp.arange(k), dtype)

def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
  preds = batched_predict(params, images)
  return -jnp.mean(preds * targets)

@jit
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]

@jit 装饰器应用于 update 函数，这是一个用于执行参数更新的函数。通常，参数更新是整个训练循环中计算密集的部分，因为它涉及计算损失梯度和参数的更新。通过使用 jit，可以将这些计算编译成高效的原生代码，加速了参数更新过程。

另一方面，accuracy 和 loss 函数似乎是用于评估模型性能和计算损失的函数，通常在每个训练周期后或在验证过程中使用。这些函数通常不是训练循环的性能瓶颈，因为它们在每个周期中只计算一次。因此，通常不需要使用 jit 来加速这些函数的执行。

In [8]:
%pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
from jax.tree_util import tree_map
from torch.utils import data
from torchvision.datasets import MNIST

def numpy_collate(batch):
  return tree_map(np.asarray, data.default_collate(batch))

class NumpyLoader(data.DataLoader):
  def __init__(self, dataset, batch_size=1,
                shuffle=False, sampler=None,
                batch_sampler=None, num_workers=0,
                pin_memory=False, drop_last=False,
                timeout=0, worker_init_fn=None):
    super(self.__class__, self).__init__(dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        sampler=sampler,
        batch_sampler=batch_sampler,
        num_workers=num_workers,
        collate_fn=numpy_collate,
        pin_memory=pin_memory,
        drop_last=drop_last,
        timeout=timeout,
        worker_init_fn=worker_init_fn)

class FlattenAndCast(object):
  def __call__(self, pic):
    return np.ravel(np.array(pic, dtype=jnp.float32))

In [10]:
# Define our dataset, using torch datasets
mnist_dataset = MNIST('/tmp/mnist/', download=True, transform=FlattenAndCast())
training_generator = NumpyLoader(mnist_dataset, batch_size=batch_size, num_workers=0)

0.3%

100.0%


Extracting /tmp/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/mnist/MNIST/raw


100.0%


Extracting /tmp/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/mnist/MNIST/raw




37.8%

100.0%
100.0%


Extracting /tmp/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/mnist/MNIST/raw

Extracting /tmp/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/mnist/MNIST/raw



In [11]:
# Get the full train dataset (for checking accuracy while training)
train_images = np.array(mnist_dataset.train_data).reshape(len(mnist_dataset.train_data), -1)
train_labels = one_hot(np.array(mnist_dataset.train_labels), n_targets)

# Get full test dataset
mnist_dataset_test = MNIST('/tmp/mnist/', download=True, train=False)
test_images = jnp.array(mnist_dataset_test.test_data.numpy().reshape(len(mnist_dataset_test.test_data), -1), dtype=jnp.float32)
test_labels = one_hot(np.array(mnist_dataset_test.test_labels), n_targets)

/home/wzhmiasanmia/miniconda3/envs/maenv/lib/python3.8/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/wzhmiasanmia/miniconda3/envs/maenv/lib/python3.8/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/wzhmiasanmia/miniconda3/envs/maenv/lib/python3.8/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/wzhmiasanmia/miniconda3/envs/maenv/lib/python3.8/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


training loop

In [12]:
import time

for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in training_generator:
    y = one_hot(y, n_targets)
    params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 5.28 sec
Training set accuracy 0.9157500267028809
Test set accuracy 0.9195999503135681
Epoch 1 in 2.93 sec
Training set accuracy 0.9372000098228455
Test set accuracy 0.9384999871253967
Epoch 2 in 2.80 sec
Training set accuracy 0.9492166638374329
Test set accuracy 0.946899950504303
Epoch 3 in 2.85 sec
Training set accuracy 0.9569166898727417
Test set accuracy 0.953499972820282
Epoch 4 in 2.78 sec
Training set accuracy 0.9629666805267334
Test set accuracy 0.9575999975204468
Epoch 5 in 2.77 sec
Training set accuracy 0.9674833416938782
Test set accuracy 0.9615999460220337
Epoch 6 in 2.81 sec
Training set accuracy 0.9708999991416931
Test set accuracy 0.9648000001907349
Epoch 7 in 2.87 sec
Training set accuracy 0.9736666679382324
Test set accuracy 0.9667999744415283
